In [68]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import matplotlib.pyplot as plt
import torchtext
import spacy
import tiktoken
import pandas as pd
import numpy as np
from gensim.corpora.dictionary import Dictionary

from tqdm.notebook import tqdm
import pickle
import ast

In [26]:
#GLOBALS

block_size = 256 #This is the value of T

In [27]:
import os
en = spacy.load('en_core_web_sm')
de = spacy.load('de_core_news_sm')
#spacy.load('en_core_web_sm')
#spacy.load('de_core_news_sm')

#dataset https://nlp.stanford.edu/projects/nmt/

In [122]:
def create_dataset(save_vocab):
    num_examples = 10000
    vocab_en = Dictionary()

    # vocab_en.add_documents([["<START>"]])
    # vocab_en.add_documents([["<END>"]])
    vocab_en.add_documents([["<PAD>"]])

    vocab_ger = Dictionary()

    vocab_ger.add_documents([["<PAD>"]])
    vocab_ger.add_documents([["<START>"]])
    vocab_ger.add_documents([["<END>"]])

    en_max = 0 
    with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
        idx_en = []
        sentences_en = []
        for i in tqdm(range(num_examples)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            doc = en(line)
            sentence = [[token.text.lower() for token in doc]]
            if len(sentence[0]) > en_max:
                en_max = len(sentence[0])
                print(en_max)
            vocab_en.add_documents(sentence)
            sentence = ["<PAD>"] + sentence[0] + ["<PAD>"]

            if len(sentence) <= block_size:
                len_pad = block_size - len(sentence)
                sentence = sentence + len_pad*"<PAD> ".split()
                assert len(sentence) == block_size, print(len(sentence))
                idx_en.append(i)
                sentences_en.append(sentence)
            else:
                sentences_en.append(np.NaN)

    print(en_max)    
    print(f"Length of sentences: {len(sentences_en)}")
    print(f"Length of english vocab: {len(vocab_en.token2id)}")

    de_max = 0 
    with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
        idx_de = []
        sentences_de = []
        for i in tqdm(range(num_examples)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            doc = de(line)
            sentence = [[token.text.lower() for token in doc]]
            if len(sentence[0]) > de_max:
                de_max = len(sentence[0])
                print(de_max)
            vocab_ger.add_documents(sentence)
            sentence = ["<START>"] + sentence[0] + ["<END>"]
            if len(sentence) <= block_size:
                len_pad = block_size - len(sentence)
                sentence = sentence + len_pad*"<PAD> ".split()
                assert len(sentence) == block_size, print(len(sentence))
                idx_de.append(i)
                sentences_de.append(sentence)
            else:
                sentences_de.append(np.NaN)
                
    print(de_max)  
    print(f"Length of sentences: {len(sentences_de)}")
    print(f"Length of german vocab: {len(vocab_de.token2id)}")

    sentences = np.array([sentences_en, sentences_de])
    idx = np.where(np.isnan(sentences))
    sentences = np.delete(test, idx[1], axis = 1)

    sentences_en = sentences[0]
    sentences_de = sentences[1]

    print(f"Length of new english sentences: {len(sentences_en)}")
    print(f"Length of new german sentences: {len(sentences_de)}")

    if save_vocab:
        print("not saving this vocab")
        vocab_en.save(os.getcwd()+"\\vocab\\vocab_en.pkl")
        vocab_ger.save(os.getcwd()+"\\vocab\\vocab_ger.pkl")

    with open(os.getcwd()+'\\data\\english_sentences.pkl', 'wb') as f:
        pickle.dump(sentences_en, f)

    with open(os.getcwd()+'\\data\\german_sentences.pkl', 'wb') as f:
        pickle.dump(sentences_de, f)

In [123]:
create = True
if create:
    create_dataset(False)
    vocab_en = Dictionary.load(os.getcwd()+"\\vocab\\vocab_en.pkl")
    vocab_de = Dictionary.load(os.getcwd()+"\\vocab\\vocab_ger.pkl")
else:
    vocab_en = Dictionary.load(os.getcwd()+"\\vocab\\vocab_en.pkl")
    vocab_de = Dictionary.load(os.getcwd()+"\\vocab\\vocab_ger.pkl")

  0%|          | 0/10000 [00:00<?, ?it/s]

32
34
47
49
54
57
58
67
87
91
92
97
98
100
105
117
130


In [76]:
s_to_i_en = vocab_en
i_to_s_en = vocab_en.token2id

s_to_i_de = vocab_de
i_to_s_de = vocab_de.token2id

In [78]:
#TRAIN AND VAL DATASETS

with open(os.getcwd()+'\\data\\english_sentences.pkl', 'rb') as f:
    english_sentences = pickle.load(f)

with open(os.getcwd()+'\\data\\german_sentences.pkl', 'rb') as f:
    german_sentences = pickle.load(f)

In [83]:
np.NaN in english_sentences

False

In [82]:
vocab_en.doc2idx(english_sentences)

TypeError: decoding to str: need a bytes-like object, list found

In [50]:
#BATCH LOADER
n = int(0.9*len(df))
train_data_x = vocab_en.doc2idx(df['inputs'].values[:n])
val_data_x = vocab_en.doc2idx(df['inputs'].values[n:])

train_data_y = vocab_de.doc2idx(df['targets'].values[:n])
val_data_y = vocab_de.doc2idx(df['targets'].values[n:])

In [52]:
train_data_x[0]

-1

rough work

In [23]:
test = [0,0,0]
[s_to_i_de[i] for i in test]

['<START>', '<START>', '<START>']

In [120]:
a = [1,2]
b = [3,np.NaN]
test = np.array([a, b])

#test = np.array([[1, 2, 3, 5], [np.NaN, 4, 3, np.NaN], [7, np.NaN, 8, 9], [np.NaN, 6, 2, np.NaN]])

test

array([[ 1.,  2.],
       [ 3., nan]])

In [121]:
sentences = np.array(test)
idx = np.where(np.isnan(sentences))
sentences = np.delete(test, idx[1], axis = 1)

sentences

array([[1.],
       [3.]])

In [115]:
idx = np.where(np.isnan(test))

In [116]:
idx[1]

array([0, 3, 1, 0, 3], dtype=int64)

In [107]:
fixed = np.delete(test, idx[1], axis = 1)

In [111]:
fixed[1]

array([3.])

In [10]:
test = "hello my name is hello name is my"
test2 = "one two three four"

doc_ = en(test)

for token in doc_:
    print(token.text.lower())
sentence_ = [[token.text for token in doc_]]
#sentence = [sentence + ['one','two']]

# sentence

sentence2 = [['one', 'two', 'hello']]

hello
my
name
is
hello
name
is
my


In [11]:
vocab = Dictionary()

vocab.add_documents(sentence_)

In [15]:
vocab_en[0]

'<PAD>'

In [ ]:
vocab.doc2idx(['hello'])

In [ ]:
vocab.save(os.getcwd()+"\\vocab\\vocab.pkl")

In [ ]:
vocab_ = Dictionary.load(os.getcwd()+"\\vocab\\vocab.pkl")

vocab_[0]
vocab_.doc2idx(['hello'])

In [ ]:
df = pd.DataFrame()

In [ ]:
vocab_en = Dictionary()

# vocab_en.add_documents([["<START>"]])
# vocab_en.add_documents([["<END>"]])
vocab_en.add_documents([["<PAD>"]])

vocab_ger = Dictionary()

vocab_ger.add_documents([["<START>"]])
vocab_ger.add_documents([["<END>"]])
vocab_ger.add_documents([["<PAD>"]])

max in german text is 238
max in english text is 342

pad both to 128

15 minutes for 100000 sentences, both german and english

328514

In [ ]:
en_max = 0 
with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
    idx_en = []
    sentences_en = []
    for i in tqdm(range(400000)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        doc = en(line)
        sentence = [[token.text.lower() for token in doc]]
        if len(sentence[0]) > en_max:
            en_max = len(sentence[0])
            print(en_max)
        vocab_en.add_documents(sentence)
        sentence = ["<PAD>"] + sentence[0] + ["<PAD>"]
        
        idx_en.append(i)
        if len(sentence) <= 256:
            len_pad = 256 - len(sentence)
            sentence = sentence + len_pad*"<PAD> ".split()
            assert len(sentence) == 256, print(len(sentence))
            sentences_en.append(sentence)
        else:
            sentences_en.append(np.NaN)

print(en_max)    
print(len(sentences_en))
print(len(vocab_en.token2id))

vocab_en.save(os.getcwd()+"\\vocab\\vocab_en.pkl")

In [ ]:
de_max = 0 
with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
    idx_de = []
    sentences_de = []
    for i in tqdm(range(400000)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        doc = de(line)
        sentence = [[token.text.lower() for token in doc]]
        if len(sentence[0]) > de_max:
            de_max = len(sentence[0])
            print(de_max)
        vocab_ger.add_documents(sentence)
        sentence = ["<START>"] + sentence[0] + ["<END>"]

        idx_de.append(i)
        if len(sentence) < 256:
            len_pad = 256 - len(sentence)
            sentence = sentence + len_pad*"<PAD> ".split()
            assert len(sentence) == 256, print(len(sentence))
            sentences_de.append(sentence)
        else:
            sentences_de.append(np.NaN)

print(de_max)  
print(len(sentences_de))
print(len(vocab_ger.token2id))

vocab_ger.save(os.getcwd()+"\\vocab\\vocab_ger.pkl")

In [ ]:
assert idx_en == idx_de, print("sentences do not align")

In [ ]:
df['idx_en'] = pd.Series(idx_en)
df['inputs'] = pd.Series(sentences_en)
df['idx_targets'] = pd.Series(idx_de)
df['targets'] = pd.Series(sentences_de)

In [ ]:
df.tail()

In [ ]:
df2 = df.loc[df['idx_en'] == df['idx_targets']]

In [ ]:
df2

In [ ]:
len(df)

In [ ]:
df = df.dropna()

In [ ]:
df = df.drop(columns = ["idx_en", "idx_targets"], axis = 1)

In [ ]:
df

In [ ]:
len(df)

In [ ]:
df.to_csv(os.getcwd()+"\\data\\data.csv", index = False)

In [ ]:
df['inputs'].values[0]

In [ ]:
vocab_en.doc2idx(df['inputs'].values[0])